### 국회의원 현황정보 수집/분석/시각화/저장
* 이름과 ID 추출
* 상세페이지 정보 추출 (json으로 저장)
* 상세정보들을 DataFrame으로 변환
* 시각화(막대그래프,히스토그램, 파이챠트, 히트맵)
* 테이블로 저장

In [14]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param_dict = {
    'currentPage':1,
    'rowPerPage':500
}
res = requests.get(url, params=req_param_dict)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select('div.memberna_list dl dt a')))
    atag_list = soup.select('div.memberna_list dl dt a')
    
    member_id_list = []
    for atag in atag_list:
        href = atag['href']
        matched = re.search(r'(\d+)', href)        
        if matched:
            member_id = matched.group(0)
        member_id_list.append(member_id)
    print(len(member_id_list))        
    print(member_id_list[:3])

200
295
295
['9771230', '9771142', '9771174']


In [39]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('===> 스크래핑 시작')
member_detail_list = []
for idx,mem_id in enumerate(member_id_list,1):
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={mem_id}'
    print(idx, detail_url)
    res = requests.get(detail_url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 정보를 저장할 dict 선언
        member_detail_dict = {}
        
        dt_list = [dt_tag.text for dt_tag in soup.select('dl.pro_detail dt')]
        
        dd_list = []        
        for dd_tag in soup.select('dl.pro_detail dd'):
            pattern = re.compile(f'[\n\r\t]')
            dd_text = pattern.sub('',dd_tag.text.strip()).replace(" ","")
            dd_list.append(dd_text)
        
        member_detail_dict = dict(zip(dt_list, dd_list))
        
        for div_tag in soup.select('div.profile'):
            #print(div_tag)
            member_detail_dict['이름']=div_tag.find('h4').text
            
            img_tag = div_tag.select('img')
            if img_tag:
                member_detail_dict['이미지'] = urljoin(detail_url, img_tag[0]['src'])
            
            member_detail_dict['생년월일'] = div_tag.select_one('li:nth-of-type(4)').text
        print(member_detail_dict)
        
        #1명의 정보가 저장된 dict를 list에 추가하기
        member_detail_list.append(member_detail_dict)

print(len(member_detail_list))        
print('===> 스크래핑 끝')        

https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd=9771230
1960-06-04
{'정당': '국민의힘', '선거구': '경남창원시성산구', '소속위원회': '보건복지위원회', '당선횟수': '재선(19대,21대)', '사무실 전화': '02-784-1751', '사무실 호실': '의원회관937호', '홈페이지': 'http://blog.naver.com/ggotop', '이메일': 'ggotop@naver.com', '보좌관': '김홍광,한영애', '비서관': '지상훈,최광림', '비서': '김영록,안효상,이유진,홍지형,김지훈', '취미, 특기': '', '의원실 안내': '', '이름': '강기윤', '이미지': 'https://www.assembly.go.kr/photo/9771230.jpg'}
https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd=9771142
1959-11-02
{'정당': '국민의힘', '선거구': '대구동구을', '소속위원회': '국방위원회,정치개혁특별위원회', '당선횟수': '초선(21대)', '사무실 전화': '02-784-5275', '사무실 호실': '의원회관341호', '홈페이지': '', '이메일': 'kds21341@naver.com', '보좌관': '박홍규,정운태', '비서관': '유진영,윤미라', '비서': '박순권,김광연,김현정,송민욱', '취미, 특기': '', '의원실 안내': '', '이름': '강대식', '이미지': 'https://www.assembly.go.kr/photo/9771142.jpg'}
